# Export EfficientDets via AutoML

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href=""><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorflow/tensorflow/lite/g3doc/models/modify/model_maker/object_detection.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Run in Azure ML Studio</a>
  </td>
</table>

### Step1. Build Environment


> * **Ubuntu Server** and **Python3.8** Compiler is required for this script. We also recommand that you can use conda to build an virtual environment before running jupyter notebook.
>   
>   ```bash
>   conda create --name automl python==3.8
>   source activate automl
>   ```

In [1]:
!pip install tensorflow-model-optimization==0.7.0
!pip install tensorflowjs==3.18.0
!pip install twine==1.9.0
!pip install urllib3==1.21.1
!pip install pandas==1.4.1
!pip install numpy==1.23.5
!pip install numba==0.53
!pip install tflite-model-maker==0.4.2
!pip install tf2onnx

### Step2. Load and Export Model Configuration

Download and extract the sample dataset for object detection, which compatible format is:
  * [Pascal_Voc](https://itriaihub.blob.core.windows.net/datasets/Detect_HardHat_244x244.pascal_voc.tar.xz)

In [2]:
from tflite_model_maker import object_detector
import numpy as np

DATASET_FOLDER_PATH = 'Detect_HardHat_244x244.pascal_voc'
CLASSES = np.load(f'{DATASET_FOLDER_PATH}/data.npy').tolist()

train_data = object_detector.DataLoader.from_pascal_voc(f'{DATASET_FOLDER_PATH}/train', f'{DATASET_FOLDER_PATH}/train', CLASSES)
valid_data = object_detector.DataLoader.from_pascal_voc(f'{DATASET_FOLDER_PATH}/test', f'{DATASET_FOLDER_PATH}/test', CLASSES)

2024-08-21 21:12:24.869353: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-21 21:12:24.869372: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/r300/miniconda3/envs/automl/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/r300/miniconda3/envs/automl/lib/python3.8/site-packages/tensorflow_

INFO:tensorflow:Cache will be stored in /tmp/tmp7x6wvb_z with prefix filename 7bee435cfee6949ada7cad38d32e815e. Cache_prefix is /tmp/tmp7x6wvb_z/7bee435cfee6949ada7cad38d32e815e
INFO:tensorflow:On image 0
INFO:tensorflow:On image 100
INFO:tensorflow:On image 200
INFO:tensorflow:Cache will be stored in /tmp/tmpgkg5fe5c with prefix filename fd42585a6188df7d3abc4e516ad0ebb5. Cache_prefix is /tmp/tmpgkg5fe5c/fd42585a6188df7d3abc4e516ad0ebb5
INFO:tensorflow:On image 0


Build Model with Training

In [3]:
from tflite_model_maker import model_spec
import tensorflow as tf

BACKBONE = 'efficientdet_lite4' # @param ["efficientdet_lite0", "efficientdet_lite1", "efficientdet_lite2", "efficientdet_lite3", "efficientdet_lite4"]

# Pre-train is necessary in AutoML Framework
model = object_detector.create(train_data, model_spec=model_spec.get(BACKBONE), epochs=1, batch_size=8,
                               train_whole_model=True, validation_data=valid_data)

INFO:tensorflow:Retraining the models...


2024-08-21 21:12:27.955551: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-08-21 21:12:27.955753: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/r300/miniconda3/envs/automl/lib/python3.8/site-packages/cv2/../../lib64:
2024-08-21 21:12:27.955821: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/r300/miniconda3/envs/automl/lib/python3.8/site-packages/cv2/../../lib64:
2024-08-21 21:12:27.955883: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcub

26/26 [==============================] - ETA: 0s - det_loss: 1.6445 - cls_loss: 1.0988 - box_loss: 0.0109 - reg_l2_loss: 0.1080 - loss: 1.7525 - learning_rate: 0.0090 - gradient_norm: 1.0497 

2024-08-21 21:18:27.056654: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


26/26 [==============================] - 327s 11s/step - det_loss: 1.6381 - cls_loss: 1.0937 - box_loss: 0.0109 - reg_l2_loss: 0.1080 - loss: 1.7461 - learning_rate: 0.0090 - gradient_norm: 1.1114 - val_det_loss: 1.4047 - val_cls_loss: 0.8334 - val_box_loss: 0.0114 - val_reg_l2_loss: 0.1080 - val_loss: 1.5127


We can preview `Params` of this model via Summary() and fill it to ITRI-AI-Hub*

In [4]:
model.summary()

Model: ""
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    multiple                  15092016  
                                                                 
 class_net/class-predict (Se  multiple                 8091      
 parableConv2D)                                                  
                                                                 
 box_net/box-predict (Separa  multiple                 10116     
 bleConv2D)                                                      
                                                                 
Total params: 15,110,223
Trainable params: 14,954,719
Non-trainable params: 155,504
_________________________________________________________________


3. Export the model and upload to [ITRI Azure Instance]() and link to [Mode Zoo]().

* **Format 1**: TFLite
(*We can preview `GFLOPS` of this model via console outputs and fill it to ITRI-AI-Hub**)

In [5]:
from tflite_model_maker.config import ExportFormat

model.export(export_dir=f'{BACKBONE}', export_format=[ExportFormat.TFLITE])

2024-08-21 21:18:35.737074: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmp1cou2lff/assets


INFO:tensorflow:Assets written to: /tmp/tmp1cou2lff/assets
2024-08-21 21:19:15.205015: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2024-08-21 21:19:26.568905: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2024-08-21 21:19:26.568930: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2024-08-21 21:19:26.569658: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp1cou2lff
2024-08-21 21:19:26.728294: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2024-08-21 21:19:26.728326: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmp1cou2lff
2024-08-21 21:19:27.237252: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel

INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmp6c4_cp3r/labelmap.txt.


INFO:tensorflow:Saving labels in /tmp/tmp6c4_cp3r/labelmap.txt.


INFO:tensorflow:TensorFlow Lite model exported successfully: efficientdet_lite4/model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: efficientdet_lite4/model.tflite


* **Format 2**: ONNX

In [6]:
!python -m tf2onnx.convert --opset 16 --tflite {BACKBONE}/model.tflite --output {BACKBONE}/model.onnx

2024-08-21 21:28:12.684487: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/r300/miniconda3/envs/automl/lib/python3.8/site-packages/cv2/../../lib64:
2024-08-21 21:28:12.684509: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/r300/miniconda3/envs/automl/lib/python3.8/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2024-08-21 21:28:14.041966: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-08-21 21:28:14.042118:

### Output differents after export (*not prepared yet)

### Benchmarking
> * You can request access to `model_zo_library` from your ITRI-AI-Hub collaborator and port the `model` and `model_zoo_library` on your device to implement the standardised test procedure.
>   
>   ```bash
>   git clone https://github.com/R300-AI/model_zoo_library.git
>   cd model_zoo_library
>
>   python3 profile.py --model <path-to-model>.tflite --platform <HW_series_name> --chipset <processor_type>
>   ```